In [3]:
import numpy as np
import os
import math
from tqdm import tqdm_notebook

dir_ = './data/'
# file_name = 'normalized_filter_track_5_user_100.csv'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [4]:
train = np.loadtxt(os.path.join(dir_, 'rating_matrix_item_' + file_name), delimiter=',')
test = np.loadtxt(os.path.join(dir_, 'test_list_item_' + file_name), delimiter=',')

In [5]:
np.set_printoptions(suppress=True)
print(train)
print(train.shape)

[[4.75 0.   0.   ... 0.   0.   0.  ]
 [2.5  0.   5.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]
(54969, 220)


In [6]:
test

array([[   71.  , 24614.  ,     0.25],
       [   33.  , 30666.  ,     1.5 ],
       [   75.  ,    33.  ,     3.  ],
       ...,
       [  127.  ,  4793.  ,     5.  ],
       [   68.  , 12003.  ,     2.25],
       [   66.  , 11733.  ,     0.25]])

In [7]:
num_tracks = len(train)
num_users = len(train[0])
print(num_users,num_tracks, len(test))

220 54969 140473


In [8]:
def pearson_correlation(tid1, tid2):
        result = 0.0
        track1_data = train[tid1]
        track2_data = train[tid2]

        rx_avg = item_average_rating(track1_data)
        ry_avg = item_average_rating(track2_data)
        sxy = common_items(track1_data, track2_data)
        
        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for user in sxy:
            rxs = track1_data[user]
            rys = track2_data[user]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return result

def item_average_rating(track):
    avg_rating = 0.0
    for i in track:
        avg_rating += i
    avg_rating /= len(track) * 1.0
    return avg_rating

def common_items(i1, i2):
    result = []
    for i in range(num_users):
        if i1[i] > 0 and i2[i] > 0:
            result.append(i)
    return result

In [9]:
def knn(tid, k):
    neighbors = []
    result = []
    for track in range(num_tracks):
        if tid == track:
            continue
        upc = pearson_correlation(tid, track)
        neighbors.append([track, upc])
    sorted_neighbors = sorted(neighbors, key=lambda neighbors: (neighbors[1], neighbors[0]), reverse=True)   # - for desc sort

    for i in range(k):
        if i >= len(sorted_neighbors):
            break
        result.append(sorted_neighbors[i])
    return result

In [10]:
def predict_by_knn(uid, tid, k_nearest_neighbors):
        valid_neighbors = check_neighbors_validattion(uid, k_nearest_neighbors)
        ##################################
        if not len(valid_neighbors):
            return -1
        ##################################
        top_result = 0.0
        bottom_result = 0.0
        for neighbor in valid_neighbors:
            neighbor_id = neighbor[0]
            neighbor_similarity = neighbor[1]   # Wi1
            rating = train[neighbor_id][uid] # rating i,item
            top_result += neighbor_similarity * rating
            bottom_result += neighbor_similarity
        result = top_result/bottom_result
        return result

def check_neighbors_validattion(uid, k_nearest_neighbors):
    result = []
    for neighbor in k_nearest_neighbors:
        neighbor_id = neighbor[0]
        if train[neighbor_id][uid] > 0:
            result.append(neighbor)
    return result


In [11]:
def predict(uid, tid, k):
    
    if len(knn_list[tid]) == 0:
        k_nearest_neighbors = knn(tid, k)
        knn_list[tid] = k_nearest_neighbors
    else:
        k_nearest_neighbors = knn_list[tid]

    prediction = predict_by_knn(uid, tid, k_nearest_neighbors)
    return prediction

In [12]:
# # test data

# test = [
#     [5,5,0,0],
#     [4,5,1,4],
#     [5,4,2,3],
#     [1,5,0,3],
#     [2,3,5,4]
# ]

# num_users = 4
# num_tracks = 5

# temp = train
# train = test

In [13]:
knn_list = []

for i in range(0, num_tracks):
    knn_list.append([])


In [ ]:
k = 30
prediction_list = []

for i in tqdm_notebook(range(len(test))):
    
    target_user = int(test[i][0])
    target_item = int(test[i][1])
    
    prediction = predict(target_user, target_item, k)
    prediction_list.append(prediction)

    print(test[i][0],test[i][1],test[i][2],prediction)

71.0 24614.0 0.25 -1
33.0 30666.0 1.5 -1
75.0 33.0 3.0 3.9
41.0 18599.0 0.25 3.25
41.0 16306.0 3.0 0.25
188.0 35455.0 3.75 1.3125000000000002
52.0 35307.0 1.25 3.5
74.0 8692.0 2.0 -1
165.0 41808.0 0.25 4.75
105.0 29611.0 4.5 3.0
61.0 26846.0 0.25 4.0
111.0 54174.0 3.25 3.75
108.0 8395.0 0.25 -1
49.0 33773.0 2.5 -1
13.0 5073.0 0.25 0.25
46.0 45590.0 4.75 -1
22.0 36521.0 4.25 -1
188.0 51382.0 0.25 1.9285714285714286
123.0 273.0 0.25 -1
190.0 251.0 0.25 -1
22.0 29354.0 3.75 -1
215.0 35560.0 4.0 -1
138.0 51333.0 0.25 2.9464285714285716
209.0 2465.0 3.0 -1
158.0 15748.0 5.0 4.75
181.0 15215.0 0.25 -1
63.0 13099.0 2.25 -1
167.0 36297.0 3.5 1.7916666666666667
74.0 41144.0 0.25 -1
7.0 18383.0 2.75 0.25
64.0 25262.0 2.5 -1
80.0 43222.0 0.25 1.125
48.0 25187.0 2.75 -1
61.0 7238.0 2.0 1.8333333333333333
194.0 28419.0 0.25 1.125
13.0 22358.0 5.0 -1
151.0 5440.0 2.0 1.8333333333333333
134.0 11549.0 4.0 0.25
55.0 36969.0 4.25 1.5833333333333333
196.0 54492.0 2.25 0.8125
169.0 11710.0 2.5 -1
127.0 33

188.0 7448.0 1.75 2.25
188.0 48463.0 1.75 1.6666666666666667
144.0 32563.0 1.75 -1
74.0 9686.0 0.25 2.8055555555555554
170.0 31957.0 1.5 3.75
88.0 22893.0 3.75 1.875
95.0 53665.0 5.0 0.25
50.0 22612.0 3.25 -1
134.0 34894.0 5.0 1.0
20.0 28177.0 0.25 -1
186.0 1542.0 2.0 -1
188.0 16869.0 4.25 1.666666666666667
73.0 10086.0 5.0 3.75
68.0 7102.0 0.25 0.25
95.0 34200.0 0.25 5.0
219.0 12424.0 3.75 1.0
130.0 16626.0 0.25 1.5833333333333333
172.0 19585.0 5.0 1.6666666666666667
211.0 444.0 4.5 2.0833333333333335
3.0 5195.0 4.75 0.25
107.0 17063.0 4.25 -1
13.0 7194.0 0.25 -1
116.0 9214.0 4.75 0.25
183.0 8884.0 3.0 -1
148.0 22502.0 1.75 2.6500000000000004
158.0 33316.0 0.25 -1
144.0 3842.0 4.25 -1
148.0 40680.0 4.5 2.5555555555555554
179.0 22952.0 2.25 2.25
188.0 3705.0 0.25 3.0
188.0 40837.0 0.25 2.25
81.0 49336.0 2.75 -1
63.0 2076.0 2.25 0.875
196.0 11291.0 4.0 0.9545454545454546
173.0 668.0 0.25 5.0
102.0 23248.0 0.25 -1
7.0 22036.0 4.25 -1
172.0 38125.0 3.75 2.875
125.0 17097.0 0.25 0.25
73.0 

46.0 35661.0 3.25 -1
45.0 4380.0 3.5 -1
107.0 48754.0 0.25 -1
124.0 22634.0 2.0 3.125
77.0 15853.0 0.25 0.25
121.0 3798.0 0.25 -1
177.0 12954.0 3.5 4.25
30.0 36252.0 0.25 -1
132.0 46418.0 1.5 3.75
211.0 53197.0 4.5 0.25
71.0 50367.0 0.25 -1
26.0 17664.0 2.75 -1
100.0 18677.0 3.5 0.25
196.0 53779.0 4.0 1.4000000000000001
149.0 31514.0 0.25 -1
8.0 15621.0 4.25 -1
93.0 43845.0 5.0 0.25
125.0 15982.0 0.25 0.25
33.0 34579.0 1.5 -1
88.0 51606.0 0.25 0.25
50.0 30039.0 4.5 -1
115.0 741.0 0.25 1.916666666666667
24.0 37037.0 0.25 -1
41.0 21053.0 5.0 1.5624999999999998
139.0 4619.0 0.25 -1
88.0 23468.0 2.25 2.6875000000000004
184.0 10522.0 2.75 1.25
151.0 8578.0 2.0 4.75
144.0 1808.0 3.5 -1
79.0 155.0 5.0 2.25
116.0 47700.0 2.0 4.333333333333333
199.0 14233.0 4.5 -1
73.0 40463.0 4.75 1.6875
167.0 45464.0 4.25 1.84375
7.0 8501.0 5.0 -1
136.0 9322.0 0.25 -1
7.0 24364.0 2.75 2.75
61.0 49591.0 0.25 -1
7.0 22373.0 0.25 -1
160.0 7209.0 5.0 -1
77.0 248.0 0.25 -1
204.0 7001.0 3.0 -1
55.0 48013.0 3.5 -1
2

165.0 13304.0 4.5 2.5
60.0 4211.0 2.0 -1
115.0 8639.0 1.5 3.75
32.0 16769.0 4.75 -1
81.0 10027.0 3.0 1.0
16.0 21268.0 3.25 -1
103.0 2320.0 3.0 -1
80.0 5004.0 0.25 -1
94.0 3873.0 2.5 -1
101.0 16669.0 3.25 -1
21.0 35329.0 5.0 -1
99.0 32001.0 0.25 0.25
44.0 16179.0 0.25 -1
133.0 53118.0 3.75 4.0
217.0 10549.0 2.5 -1
158.0 678.0 3.5 3.9166666666666665
102.0 37622.0 0.25 -1
200.0 30471.0 1.25 4.0
147.0 53533.0 3.5 -1
111.0 2111.0 3.25 0.25
211.0 2242.0 4.0 0.25
172.0 12472.0 2.5 2.025
100.0 10112.0 4.75 -1
195.0 15245.0 4.5 2.375
204.0 31607.0 0.25 -1
183.0 38094.0 0.25 -1
37.0 477.0 4.0 -1
210.0 18684.0 0.25 1.6666666666666667
34.0 1888.0 0.25 -1
81.0 368.0 4.25 -1
182.0 4301.0 0.25 2.015625
115.0 15032.0 4.25 3.416666666666667
150.0 48871.0 0.25 2.6250000000000004
171.0 32548.0 3.5 1.875
148.0 4251.0 1.75 2.5
43.0 41813.0 0.25 -1
198.0 31066.0 5.0 -1
163.0 7254.0 0.25 -1
207.0 2732.0 4.25 1.75
151.0 13514.0 3.25 2.0
147.0 27859.0 0.25 0.9
7.0 3918.0 4.25 -1
81.0 326.0 0.75 2.375
165.0 715

116.0 1963.0 4.5 2.5
33.0 27104.0 1.5 3.75
105.0 17399.0 0.25 2.9166666666666665
80.0 41315.0 0.25 -1
76.0 24287.0 5.0 0.25
185.0 3050.0 3.5 -1
196.0 51758.0 0.25 0.9545454545454547
52.0 7737.0 2.25 -1
50.0 35034.0 2.75 1.75
0.0 1180.0 3.0 -1
10.0 10103.0 0.25 0.25
68.0 807.0 4.5 -1
61.0 5128.0 3.5 3.5
178.0 26630.0 4.25 0.25
116.0 12287.0 1.25 4.5
73.0 4064.0 1.5 2.583333333333333
100.0 7012.0 3.5 0.25
211.0 34925.0 0.25 2.375
203.0 32765.0 0.25 2.75
178.0 44892.0 4.0 2.78125
196.0 52523.0 2.25 2.21875
4.0 13015.0 0.25 -1
169.0 41531.0 3.25 2.75
88.0 24604.0 4.5 0.25
80.0 50435.0 2.0 -1
110.0 13253.0 3.5 -1
68.0 15134.0 3.75 -1
148.0 39655.0 3.25 1.4500000000000002
147.0 15279.0 0.25 0.25
15.0 4444.0 0.25 -1
202.0 2679.0 2.75 3.25
32.0 27374.0 2.5 -1
143.0 18722.0 2.25 2.375
74.0 28728.0 5.0 -1
194.0 41947.0 0.25 1.4166666666666665
55.0 10928.0 1.5 0.25
39.0 36703.0 0.25 -1
161.0 12999.0 0.25 -1
62.0 2563.0 2.75 -1
147.0 17675.0 3.5 0.25
194.0 2102.0 0.25 0.95
193.0 22883.0 0.25 2.250

...

61.0 16892.0 4.5 1.8333333333333333
167.0 13644.0 0.25 2.1250000000000004
121.0 53152.0 0.25 0.25
89.0 10759.0 2.0 -1
13.0 27027.0 4.75 -1
148.0 30088.0 0.25 2.140625
24.0 26674.0 4.5 -1
62.0 39500.0 0.25 -1
182.0 20628.0 0.25 2.25
148.0 23776.0 4.0 1.8750000000000002
147.0 54554.0 3.5 0.25
172.0 5421.0 0.25 1.625
172.0 34133.0 3.5 1.25
38.0 13949.0 0.25 -1
130.0 27440.0 2.5 0.25
77.0 35278.0 1.5 3.25
148.0 34113.0 3.25 2.267857142857143
4.0 13433.0 4.25 1.5833333333333333
61.0 43599.0 4.25 0.25
205.0 2940.0 5.0 -1
119.0 44594.0 0.25 0.25
148.0 13313.0 1.75 0.9791666666666666
50.0 47417.0 0.25 -1
81.0 15793.0 4.25 2.5
167.0 4015.0 0.25 0.25
88.0 19738.0 2.25 3.041666666666667
19.0 12467.0 4.25 1.75
179.0 7318.0 0.25 0.25
151.0 46747.0 2.0 0.8333333333333334
116.0 37332.0 5.0 -1
138.0 40350.0 4.75 -1
43.0 4795.0 0.25 0.25
60.0 7927.0 3.25 -1
24.0 33999.0 3.25 -1
39.0 10678.0 1.25 -1
80.0 14298.0 4.25 -1
72.0 4143.0 0.25 1.833333333333333
192.0 29367.0 0.25 3.75
183.0 12111.0 4.25 0.25
4

73.0 11041.0 3.75 1.25
147.0 40045.0 0.25 1.3333333333333333
103.0 53947.0 3.0 4.0
50.0 10653.0 2.75 1.5714285714285718
159.0 7446.0 3.75 -1
124.0 40417.0 1.25 0.25
171.0 42224.0 4.0 0.9
148.0 7850.0 4.0 2.5000000000000004
130.0 3362.0 4.0 -1
174.0 13289.0 0.25 2.5
183.0 15482.0 0.25 3.625
61.0 7674.0 3.75 2.0
188.0 43079.0 3.75 2.375
3.0 8117.0 4.0 -1
193.0 14526.0 1.75 5.0
172.0 11597.0 4.75 2.6666666666666665
71.0 9846.0 4.0 -1
167.0 18036.0 4.5 1.375
2.0 4241.0 0.25 -1
155.0 16577.0 2.0 1.7
13.0 28898.0 0.25 -1
194.0 13565.0 3.75 0.25
7.0 14498.0 4.25 1.0833333333333333
148.0 48212.0 0.25 2.916666666666667
196.0 19944.0 5.0 2.178571428571429
202.0 15689.0 0.25 -1
179.0 6186.0 2.25 0.25
214.0 10469.0 0.25 4.75
179.0 14903.0 0.25 0.25
196.0 54463.0 0.25 1.0178571428571428
141.0 2179.0 0.25 -1
0.0 1674.0 4.25 0.25
95.0 7416.0 2.5 4.0
41.0 44519.0 3.0 -1
159.0 15066.0 0.25 -1
21.0 24674.0 0.25 -1
71.0 3067.0 5.0 -1
33.0 41344.0 0.25 -1
105.0 3839.0 4.75 0.25
19.0 34302.0 0.25 -1
182.0 

178.0 44676.0 0.25 0.25
147.0 54813.0 3.5 2.2500000000000004
195.0 16589.0 0.25 1.75
120.0 52800.0 3.5 -1
183.0 12457.0 0.25 -1
151.0 5744.0 0.25 2.75
188.0 41977.0 0.25 3.0277777777777777
126.0 17723.0 0.25 -1
209.0 9933.0 2.25 2.5
210.0 1738.0 2.25 2.125
52.0 43774.0 0.25 1.125
193.0 54073.0 0.25 -1
194.0 15222.0 0.25 0.25
174.0 15482.0 0.25 2.5833333333333335
147.0 20798.0 0.25 0.25
147.0 3040.0 0.25 1.55
108.0 16667.0 0.25 -1
171.0 43678.0 0.25 2.0
167.0 28514.0 0.25 3.4375
161.0 54520.0 3.5 -1
196.0 39749.0 4.0 0.65
189.0 7229.0 0.75 2.5
111.0 40282.0 4.25 -1
148.0 1321.0 3.25 1.9545454545454546
4.0 12354.0 0.25 -1
77.0 7234.0 4.5 2.3125
73.0 9271.0 4.0 2.125
116.0 3989.0 4.0 -1
208.0 40914.0 4.0 4.25
51.0 22234.0 0.25 -1
100.0 1822.0 4.25 3.5
61.0 4652.0 3.0 -1
33.0 16521.0 3.25 -1
125.0 41924.0 4.0 -1
4.0 10401.0 0.25 -1
3.0 2298.0 3.5 -1
148.0 45745.0 1.75 2.115384615384616
182.0 18793.0 0.25 1.1249999999999998
148.0 20472.0 1.75 2.0384615384615383
73.0 39000.0 1.5 0.25
155.0 5

191.0 27234.0 1.25 -1
178.0 2670.0 4.25 3.25
194.0 18547.0 0.25 0.25
148.0 21111.0 0.25 1.4772727272727273
57.0 31931.0 1.25 -1
172.0 6509.0 3.0 0.8750000000000002
12.0 1878.0 0.25 2.9000000000000004
148.0 9131.0 0.25 1.9375
6.0 16538.0 2.5 -1
0.0 2626.0 1.5 -1
24.0 37234.0 0.25 -1
132.0 18862.0 0.25 -1
59.0 26240.0 0.25 -1
174.0 41870.0 3.0 2.0000000000000004
13.0 26078.0 4.25 1.625
126.0 885.0 3.0 -1
107.0 24362.0 4.5 -1
182.0 18161.0 1.75 2.375
188.0 17828.0 3.75 3.0833333333333335
188.0 31932.0 4.25 3.2142857142857144
4.0 11918.0 0.25 -1
100.0 13117.0 5.0 -1
116.0 13145.0 3.75 2.625
164.0 21637.0 3.75 -1
73.0 22310.0 0.25 3.583333333333333
149.0 14058.0 0.25 0.25
66.0 36619.0 0.25 -1
123.0 23897.0 0.25 2.9999999999999996
41.0 20600.0 0.25 -1
6.0 5180.0 0.25 -1
100.0 35872.0 3.5 -1
196.0 39745.0 2.25 1.833333333333333
40.0 42702.0 0.25 -1
60.0 31844.0 0.25 2.95
167.0 47359.0 2.75 1.6590909090909092
71.0 36954.0 0.25 0.25
57.0 18011.0 3.25 -1
167.0 1118.0 0.25 2.0277777777777777
211.

151.0 6070.0 5.0 2.416666666666667
130.0 11168.0 4.0 0.25
196.0 47429.0 0.25 1.4090909090909092
178.0 25803.0 3.25 0.875
161.0 33971.0 2.25 -1
60.0 1750.0 1.25 -1
19.0 7998.0 4.25 -1
102.0 31846.0 0.25 -1
167.0 5250.0 0.25 2.7
64.0 13733.0 0.25 2.5
76.0 15023.0 0.25 3.75
174.0 32358.0 0.25 2.625
107.0 25958.0 2.5 -1
130.0 2689.0 0.25 2.5
74.0 33254.0 3.0 -1
182.0 11275.0 0.25 1.7142857142857142
151.0 35070.0 0.25 -1
155.0 8566.0 5.0 1.5833333333333333
171.0 8637.0 2.0 2.28125
22.0 36159.0 2.25 -1
123.0 42529.0 2.0 0.25
138.0 49060.0 0.25 -1
196.0 22645.0 2.25 1.8
166.0 26442.0 0.25 2.0
74.0 9561.0 4.25 1.125
198.0 13546.0 2.75 0.25
172.0 37529.0 5.0 2.0
10.0 26381.0 0.25 -1
72.0 42521.0 0.25 2.0000000000000004
167.0 6906.0 5.0 0.25
143.0 19659.0 2.25 -1
171.0 23383.0 2.0 2.7500000000000004
72.0 24336.0 2.75 0.8749999999999999
148.0 52768.0 4.0 2.125
179.0 4545.0 2.25 -1
205.0 534.0 3.0 -1
207.0 444.0 4.5 0.25
208.0 6869.0 0.25 2.75
100.0 10101.0 3.5 2.25
219.0 33218.0 1.75 4.75
50.0 13

In [ ]:
np.savetxt(os.path.join(dir_, 'prediction__item_' + file_name), prediction_list, delimiter=",")

In [ ]:
dict ={}
dict[str(1)+','+str(1)]=0.656
dict[str(1)+','+str(1)]
if str(1)+','+str(2) in dict:
    print(1)
else:
    print(2)